# Description

The goal of this notebook is very simple: it just lists the content (traits/diseases) that belong to each cluster across all selected "best partitions". Although one would take a look at them here to check whether clusters of traits make sense, that analysis is carried out first by looking at the clustering trees (which are generated later). Then, this notebooks serves as a simple list with the content of the clusters.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

from utils import generate_result_set_name
import conf

# Settings

In [3]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering')

## Load data

In [4]:
INPUT_SUBSET = "umap"

In [5]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [6]:
DR_OPTIONS = {
    "n_components": 5,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [7]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0'

In [8]:
data_umap = pd.read_pickle(input_filepath)

In [9]:
data_umap.shape

(3752, 5)

In [10]:
data_umap.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
100001_raw-Food_weight,0.426554,0.670532,7.363805,1.171837,6.297295
100002_raw-Energy,-1.605179,0.815699,8.288521,0.990394,6.817351
100003_raw-Protein,-1.656178,0.788297,8.355906,1.017072,6.845651
100004_raw-Fat,-1.508325,0.802536,8.328274,1.033939,6.709319
100005_raw-Carbohydrate,-1.617872,0.812711,8.307973,1.020575,6.825944


# Load best partitions

In [11]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/best_partitions_by_k.pkl')

In [12]:
best_partitions = pd.read_pickle(input_file)

In [13]:
best_partitions.shape

(59, 4)

In [14]:
best_partitions.head()

,method,partition,ari_median,selected
k,,,,
11,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.090231,True
13,scc_025,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.090222,True
14,scc_025,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.090138,True
22,scc_020,"[1, 19, 19, 19, 19, 19, 19, 19, 19, 1, 19, 19,...",0.090117,True
12,scc_025,"[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...",0.089742,True


# Analysis of clusterings

In [15]:
from IPython.display import HTML

In [16]:
def show_cluster_stats(clustering_data, selected_partition, selected_cluster):
    traits = [t for t in clustering_data[selected_partition == selected_cluster].index]
    display(len(traits))
    display(traits)

In [17]:
selected_k_values = best_partitions[best_partitions["selected"]].index.tolist()
selected_k_values.sort()
display(selected_k_values)

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 22, 25, 26]

In [18]:
for k in selected_k_values:
    display(HTML(f"<h2>Partition with k={k}</h2>"))
    display(best_partitions.loc[k])

    part = best_partitions.loc[k, "partition"]
    display(part.shape)

    part_stats = pd.Series(part).value_counts()
    display(part_stats)

    for cluster_number in part_stats.index.sort_values():
        display(HTML(f"<h3>Cluster {k}.{cluster_number}</h3>"))

        cluster_traits = data_umap[part == cluster_number].index
        display(len(cluster_traits))
        display(cluster_traits)

method                                                  scc_030
partition     [1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 4, 4, 1, 1, 1, ...
ari_median                                            0.0876165
selected                                                   True
Name: 5, dtype: object

(3752,)

1    3233
4     445
0      47
3      15
2      12
dtype: int64

47

Index(['20015_raw-Sitting_height',
       '20150_raw-Forced_expiratory_volume_in_1second_FEV1_Best_measure',
       '20151_raw-Forced_vital_capacity_FVC_Best_measure',
       '20153_raw-Forced_expiratory_volume_in_1second_FEV1_predicted',
       '21001_raw-Body_mass_index_BMI', '21002_raw-Weight', '23098_raw-Weight',
       '23099_raw-Body_fat_percentage', '23100_raw-Whole_body_fat_mass',
       '23101_raw-Whole_body_fatfree_mass', '23102_raw-Whole_body_water_mass',
       '23104_raw-Body_mass_index_BMI', '23105_raw-Basal_metabolic_rate',
       '23106_raw-Impedance_of_whole_body', '23107_raw-Impedance_of_leg_right',
       '23108_raw-Impedance_of_leg_left', '23109_raw-Impedance_of_arm_right',
       '23110_raw-Impedance_of_arm_left', '23111_raw-Leg_fat_percentage_right',
       '23112_raw-Leg_fat_mass_right', '23113_raw-Leg_fatfree_mass_right',
       '23114_raw-Leg_predicted_mass_right',
       '23115_raw-Leg_fat_percentage_left', '23116_raw-Leg_fat_mass_left',
       '23117_raw-Leg_

...

Index(['21001_raw-Body_mass_index_BMI', '21002_raw-Weight', '23098_raw-Weight',
       '23099_raw-Body_fat_percentage', '23100_raw-Whole_body_fat_mass',
       '23104_raw-Body_mass_index_BMI', '23111_raw-Leg_fat_percentage_right',
       '23112_raw-Leg_fat_mass_right', '23115_raw-Leg_fat_percentage_left',
       '23116_raw-Leg_fat_mass_left', '23119_raw-Arm_fat_percentage_right',
       '23120_raw-Arm_fat_mass_right', '23123_raw-Arm_fat_percentage_left',
       '23124_raw-Arm_fat_mass_left', '23127_raw-Trunk_fat_percentage',
       '23128_raw-Trunk_fat_mass', '48_raw-Waist_circumference',
       '49_raw-Hip_circumference'],
      dtype='object')

1

Index(['30070_raw-Red_blood_cell_erythrocyte_distribution_width'], dtype='object')

101

Index(['41248_1000-Destinations_on_discharge_from_hospital_recoded_Usual_Place_of_residence',
       '4194_raw-Pulse_rate', '4260-Round_of_numeric_memory_test',
       '4282-Maximum_digits_remembered_correctly',
       '4283-Number_of_rounds_of_numeric_memory_test_performed',
       '4290_raw-Duration_screen_displayed', '4291-Number_of_attempts',
       '46_raw-Hand_grip_strength_left', '47_raw-Hand_grip_strength_right',
       '4825-Noisy_workplace',
       ...
       '864-Number_of_daysweek_walked_10_minutes', '874_raw-Duration_of_walks',
       '884-Number_of_daysweek_of_moderate_physical_activity_10_minutes',
       '894-Duration_of_moderate_activity',
       '904-Number_of_daysweek_of_vigorous_physical_activity_10_minutes',
       '924-Usual_walking_pace',
       '943-Frequency_of_stair_climbing_in_last_4_weeks',
       '971-Frequency_of_walking_for_pleasure_in_last_4_weeks',
       'ICDMAIN_ANY_ENTRY-Any_ICDMAIN_event_in_hilmo_or_causes_of_death',
       'episodic memory'],
     